In [3]:
from random import randint, uniform, choice
from util import generate_problem, verify_witness

INF = 1e9

## Firstly we need a solver for the STP problem.

note here we are given a distance graph
(the generation is done in the genetic part, so we can reuse memory and save time when an interval selection is changed.)

In [4]:
def discrete_graph(N):
    mat = [[INF for _ in range(N)] for _ in range(N)]
    for i in range(N):
        mat[i][i] = 0
    return mat

def generate_d_graph(graph):
    N = len(graph)
    E = discrete_graph(N)
    
    for i in range(N):
        E[i][i] = 0 # redundant

    for i in range(N):
        for j in range(N):
            E[i][j] = graph[i][j]

    for k in range(N):
        for i in range(N):
            for j in range(N):
                E[i][j] = min(E[i][j], E[i][k] + E[k][j])

    return E

def get_min_sol(d_graph):
    return [-d_graph[i][0] for i in range(len(d_graph))]

def consistent(d_graph):
    for i in range(len(d_graph)):
        if d_graph[i][i] < 0: return False
    return True

def solve_stp(graph):
    ''' graph is the constraint adjacency matrix '''
    
    d_graph = generate_d_graph(graph)
    
    return consistent(d_graph), get_min_sol(d_graph)

## The genetic part:

In [52]:
def update_graph(gene):
    # set up graph using index 0 everywhere
    # so later we can change interval in O(1) time
    # (saves memory and time, and this same approach is especially beneficial in meta_walk)
    
    interval_selection = gene[0]
    graph = gene[1]
    gene[2] = None # unset
    
    for i, constr in enumerate(T):
        interval_index = interval_selection[i]
        
        i, j = constr['i'], constr['j']
        interval = constr['intervals'][interval_index]
        
        graph[i][j] = interval[1]
        graph[j][i] = -interval[0]

def update_graph_at_constraint(gene, constraint_index):
    ''' update_graph but updates only one constraint '''
    
    interval_selection = gene[0]
    graph = gene[1]
    gene[2] = None # unset
    
    interval_index = interval_selection[constraint_index]
    constr = T[constraint_index]
    
    i, j = constr['i'], constr['j']
    interval = constr['intervals'][interval_index]

    graph[i][j] = interval[1]
    graph[j][i] = -interval[0]

def random_gene(T):
    num_variables = max([max(t['i'], t['j']) for t in T])
    interval_selection = [randint(0, len(constr['intervals'])-1) for constr in T]
    gene = [interval_selection, discrete_graph(num_variables+1), None]
    update_graph(gene)
    d_graph = generate_d_graph(gene[1]) # costly op.
    failed = verify_witness(get_min_sol(d_graph), T)
    gene[2] = failed
    
    return gene
    
def walk_gene(gene, T):
    # ignore all that have no choice
    # create list of plausible indices and filter it.
    candidate_indices = [j for j in range(len(T))]
    candidate_indices = list(filter(
        lambda x: len(T[x]['intervals']) > 1,
        candidate_indices
    ))
    
    idx = choice(candidate_indices)
    gene[0][idx] = randint(0, len(T[idx]['intervals'])-1)
    update_graph_at_constraint(gene, idx)

def meta_walk(T, max_iterations, max_flips):
    num_variables = max([max(t['i'], t['j']) for t in T])
    graph = discrete_graph(num_variables+1)
    
    best_gene = None
    best_gene_failed = None
    
    for i in range(max_iterations):
        gene = random_gene(T)
        update_graph(gene)
        
        for j in range(max_flips):
            # at each iteration we modify

            is_consistent, witness = solve_stp(gene[1])
            gene_failed = verify_witness(witness, T)
            gene[2] = gene_failed
            
            if not best_gene or len(gene_failed) < len(best_gene_failed):
                best_gene = gene
                best_gene_failed = gene_failed

            if is_consistent:
                break
                
            walk_gene(gene, T)
        
    print(f'num flips: {i+1}, num iterations: {j+1}')
    print('best gene:', best_gene)
    print('constraints failed:', len(best_gene_failed), 'out of:', len(T)) 
    print('failed constraints:', best_gene_failed)

In [64]:
T = generate_problem()
meta_walk(T, 1, 50)

num flips: 1, num iterations: 50
best gene: [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 2], [[100, 100, 100, 100, 100, 100, 100, 100, 100, 100], [100, 0, -10, 1000000000.0, 1000000000.0, 78, 1000000000.0, -78, 1000000000.0, 1000000000.0], [100, 26, 0, 1000000000.0, -59, 1000000000.0, 1000000000.0, 1000000000.0, 1000000000.0, 23], [100, 1000000000.0, 1000000000.0, 0, 1000000000.0, 39, 1000000000.0, 1000000000.0, 1000000000.0, 1000000000.0], [100, 1000000000.0, 70, 1000000000.0, 0, 1000000000.0, -83, 19, -59, 1000000000.0], [100, -29, 1000000000.0, 70, 1000000000.0, 0, 1000000000.0, 1000000000.0, 1000000000.0, 1000000000.0], [100, 1000000000.0, 1000000000.0, 1000000000.0, 83, 1000000000.0, 0, 16, 1000000000.0, 1000000000.0], [100, 96, 1000000000.0, 1000000000.0, 69, 1000000000.0, -12, 0, 73, 1000000000.0], [100, 1000000000.0, 1000000000.0, 1000000000.0, 70, 1000000000.0, 1000000000.0, 35, 0, 1000000000.0], [100, 1000000000.0, 85, 1000000000.0, 1000000000.0, 1000000000.0

## Graphing effectiveness of walking a gene 

## WARNING: (TODO)
###  (it is hard to say if this proves anything -- it's basically the same as random)
### however, this can be analysed on a more larger-scale
### eg. walking is faster than regenerating the whole gene and graph





Maybe something can be said about performance and complexity of random-random-random-random...
vs random-walk-walk-walk...

In [66]:
# problem sets:
SIMPLE = {
    'constraint_probability': 0.25,
    'min_intervals': 1,
    'max_intervals': 1,
    'scaling_factor': 1,
}

BINARY = {
    'constraint_probability': 0.25,
    'min_intervals': 2,
    'max_intervals': 2,
    'scaling_factor': 1,
}

FIVE_MAX = {
    'constraint_probability': 0.25,
    'min_intervals': 1,
    'max_intervals': 5,
    'scaling_factor': 1,
}

In [72]:
runs = 50
run_results = []

def test(T, max_flips):
    unsats=[]
    
    gene = random_gene(T)
    
    for i in range(max_flips):
        is_consistent, witness = solve_stp(gene[1])
        gene_failed = verify_witness(witness, T)
        gene[2] = gene_failed
        unsats += [len(gene_failed)]

        if is_consistent: break

        walk_gene(gene, T)
        
    return unsats


for _ in range(runs):
    T = generate_problem(
        variables=20,
        **BINARY,
    )
    
    arr = test(T, max_flips=100)
    starting = arr[0]
    best = min(arr)
    if arr[0] > 0:
        decrease = (arr[0]-best) / arr[0]
    else:
        decrease = 1.0
    
    # TODO, problem: we do not know if it is even solvable!
    # will have to confirm with backtracking, or make sure to generate only solvable problems
    solved = best == 0
    
    print(starting, best, f'{decrease:.2f}', solved)

43 36 0.16 False
55 55 0.00 False
22 22 0.00 False
41 40 0.02 False
45 44 0.02 False
46 43 0.07 False
55 38 0.31 False
52 49 0.06 False
55 55 0.00 False
52 36 0.31 False
50 50 0.00 False
41 37 0.10 False
38 34 0.11 False
28 23 0.18 False
33 26 0.21 False
43 41 0.05 False
66 54 0.18 False
55 55 0.00 False
57 54 0.05 False
53 52 0.02 False
44 37 0.16 False
57 44 0.23 False
58 55 0.05 False
51 49 0.04 False
58 55 0.05 False
36 24 0.33 False
53 43 0.19 False
63 48 0.24 False
40 33 0.17 False
52 48 0.08 False
42 41 0.02 False
49 41 0.16 False
56 46 0.18 False
41 40 0.02 False
56 50 0.11 False
47 41 0.13 False
62 59 0.05 False
47 29 0.38 False
26 25 0.04 False
56 39 0.30 False
55 52 0.05 False
42 37 0.12 False
51 48 0.06 False
56 44 0.21 False
43 42 0.02 False
16 12 0.25 False
42 39 0.07 False
48 48 0.00 False
44 44 0.00 False
56 56 0.00 False
